In [3]:
import pandas as pd

In [8]:
!pip install pyspark --> None

# Домашнее задание
Условие: дана таблица с колонками id, name, salary, managerID. Необходимо написать код на spark, который создаст эту таблицу и в рез-те выдаст таблицу, в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров.

In [11]:
import pyspark

In [14]:
from pyspark.sql import SparkSession

In [15]:
spark = SparkSession.builder.appName('hw').getOrCreate()

In [28]:
from pyspark.sql.functions import col

In [43]:
data = {'id': [1, 2, 3, 4], 'Name': ['Joe', 'Jennifer', 'Sam', 'Mary'],
        'Salary': ['70', '80', '60', '90'], 'ManagerId': ['3', '4', 'Null', 'Null']}
df = pd.DataFrame(data)
df

,id,Name,Salary,ManagerId
0,1,Joe,70,3
1,2,Jennifer,80,4
2,3,Sam,60,Null
3,4,Mary,90,Null


In [44]:
df = spark.createDataFrame(df)
df

DataFrame[id: bigint, Name: string, Salary: string, ManagerId: string]

In [45]:
df = df.withColumn("Salary", df["Salary"].cast("integer"))
df

DataFrame[id: bigint, Name: string, Salary: int, ManagerId: string]

In [35]:
joined_df = df.alias("emp").join(df.alias("mgr"), col("emp.ManagerId") == col("mgr.id"), how="left")
joined_df.show()

+---+--------+------+---------+----+----+------+---------+
| id|    Name|Salary|ManagerId|  id|Name|Salary|ManagerId|
+---+--------+------+---------+----+----+------+---------+
|  1|     Joe|    70|        3|   3| Sam|    60|     Null|
|  2|Jennifer|    80|        4|   4|Mary|    90|     Null|
|  3|     Sam|    60|     Null|NULL|NULL|  NULL|     NULL|
|  4|    Mary|    90|     Null|NULL|NULL|  NULL|     NULL|
+---+--------+------+---------+----+----+------+---------+



In [37]:
res_df = joined_df.filter(col("emp.Salary") > col("mgr.Salary")).select(col("emp.Name")).alias("Employee")
res_df.show()

+----+
|Name|
+----+
| Joe|
+----+

